In [1]:
import pandas as pd
from pymongo import MongoClient
from collections import defaultdict
from urllib.parse import quote_plus
import requests
from pandas_datareader import wb
from pandas_datareader import data as pdr
from pandasdmx import Request

D:\Archivos_de_programas\envs\PERSONAL_PYTHON\lib\site-packages\pandasdmx\remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


In [2]:
# Paso 1: extraer tasa de urbanización (SP.URB.TOTL.IN.ZS) desde WB
from pandas_datareader import wb
import pandas as pd

# Parámetros
countries = ['ES', 'DE', 'IT', 'SE']
start_year = 2000
end_year   = 2025

# Descarga
df_urb = wb.download(
    indicator='SP.URB.TOTL.IN.ZS',
    country=countries,
    start=start_year,
    end=end_year
)

# Formatear
df_urb = (
    df_urb
      .reset_index()                       # convierte MultiIndex a columnas
      .rename(columns={
          'country': 'geo',
          'date':    'time_period',
          'SP.URB.TOTL.IN.ZS': 'urban_rate'
      })
)

C:\Users\jalop\AppData\Local\Temp\ipykernel_26212\316849397.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_urb = wb.download(


In [3]:
# Comprobar
print(df_urb.head(8))
print("\nGeos:", df_urb['geo'].unique())

       geo  year  urban_rate
0  Germany  2024         NaN
1  Germany  2023      77.765
2  Germany  2022      77.648
3  Germany  2021      77.544
4  Germany  2020      77.453
5  Germany  2019      77.376
6  Germany  2018      77.312
7  Germany  2017      77.261

Geos: ['Germany' 'Spain' 'Italy' 'Sweden']


In [4]:
# 1) Renombrar year → time_period
df_urb = df_urb.rename(columns={'year':'time_period'})

# 2) Mapear nombres completos a ISO2
name_to_iso = {
    'Germany': 'DE',
    'Spain':   'ES',
    'Italy':   'IT',
    'Sweden':  'SE'
}
df_urb['geo'] = df_urb['geo'].replace(name_to_iso)

# 3) Asegurarnos de que time_period es entero
df_urb['time_period'] = df_urb['time_period'].astype(int)

# 4) Vista rápida
print(df_urb.head(8))
print("\nGeos únicos:", sorted(df_urb['geo'].unique()))
print("Años únicos:", sorted(df_urb['time_period'].unique())[:5], "...", sorted(df_urb['time_period'].unique())[-5:])

  geo  time_period  urban_rate
0  DE         2024         NaN
1  DE         2023      77.765
2  DE         2022      77.648
3  DE         2021      77.544
4  DE         2020      77.453
5  DE         2019      77.376
6  DE         2018      77.312
7  DE         2017      77.261

Geos únicos: ['DE', 'ES', 'IT', 'SE']
Años únicos: [np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004)] ... [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]


In [15]:
df_raw = pd.read_csv("../data/gasto_publico/inclusion_findex.csv", delimiter=";")

In [16]:
df_raw

,Country Name,Country Code,Series Name,Series Code,2011 [YR2011],2014 [YR2014],2017 [YR2017],2021 [YR2021]
0,Spain,ESP,Account (% age 15+),account.t.d,93.28,97.58,93.76,98.30
1,Spain,ESP,Made or received a digital payment (% age 15+),g20.t.d,NaN,94.20,90.49,97.53
2,Germany,DEU,Account (% age 15+),account.t.d,98.13,98.76,99.14,99.98
3,Germany,DEU,Made or received a digital payment (% age 15+),g20.t.d,NaN,95.80,97.76,99.48
4,Italy,ITA,Account (% age 15+),account.t.d,71.01,87.33,93.79,97.29
5,Italy,ITA,Made or received a digital payment (% age 15+),g20.t.d,NaN,73.24,89.66,95.78
6,Sweden,SWE,Account (% age 15+),account.t.d,98.99,99.72,99.74,99.69
7,Sweden,SWE,Made or received a digital payment (% age 15+),g20.t.d,NaN,99.21,98.34,99.06


In [49]:
# 2) Renombrar columnas de año y quedarnos con años útiles
df_raw = df_raw.rename(columns={
    "2011 [YR2011]": "2011",
    "2014 [YR2014]": "2014",
    "2017 [YR2017]": "2017",
    "2021 [YR2021]": "2021"
})

# 3) Filtrar solo columnas relevantes
df = df_raw[["Country Code", "Series Name", "2011", "2014", "2017", "2021"]]

# 4) Reorganizar a formato largo
df = df.melt(id_vars=["Country Code", "Series Name"], var_name="time_period", value_name="value")

# 5) Pivot para que cada fila tenga ambas métricas
df = df.pivot_table(
    index=["Country Code", "time_period"],
    columns="Series Name",
    values="value"
).reset_index()

# 6) Renombrar columnas clave
df = df.rename(columns={
    "Country Code": "geo",
    "Account (% age 15+)": "account_pct",
    "Made or received a digital payment (% age 15+)": "digital_payments_pct"
})

# 7) Convertir a numérico y calcular índice medio
df["account_pct"] = pd.to_numeric(df["account_pct"], errors="coerce")
df["digital_payments_pct"] = pd.to_numeric(df["digital_payments_pct"], errors="coerce")
df["inclusion_index"] = df[["account_pct", "digital_payments_pct"]].mean(axis=1)

# 8) Asegurar tipos
df["time_period"] = df["time_period"].astype(int)

# 9) Vista final
print(df.head())

Series Name  geo  time_period  account_pct  digital_payments_pct  \
0            DEU         2011        98.13                   NaN   
1            DEU         2014        98.76                 95.80   
2            DEU         2017        99.14                 97.76   
3            DEU         2021        99.98                 99.48   
4            ESP         2011        93.28                   NaN   

Series Name  inclusion_index  
0                      98.13  
1                      97.28  
2                      98.45  
3                      99.73  
4                      93.28  


In [50]:
# Configurar conexión
usuario = "jalope"
contrasena = "admin"
host = "127.0.0.1"
puerto = "27250"

uri = f"mongodb://{quote_plus(usuario)}:{quote_plus(contrasena)}@{host}:{puerto}/?directConnection=true"
client = MongoClient(uri)
db = client["tfm_db"]
collection = db['EUROSTAT_SPENDING_FLOWS']

# 2. Leer todos los documentos
docs = list(collection.find({}))

# 3. Convertir a DataFrame
df_merged = pd.DataFrame(docs)

# 4. Asegurar que time_period es int
df_merged['time_period'] = df_merged['time_period'].astype(int)

# 5. Vista rápida de columnas
print("Columnas en df_merged:", df_merged.columns.tolist())

Columnas en df_merged: ['_id', 'geo', 'time_period', 'gf01_general_public_services', 'gf07_health', 'gf09_education', 'gf1002_old_age', 'd51c1', 'd51c2', 'd51c3', 'd51d', 'd51e', 'd59', 'd61', 'd91', 'inclusion_index']


In [54]:
df_merged = df_merged.drop('inclusion_index', axis=1)

In [55]:
# 1. Asegurar que la columna geo coincide con ISO-2 (ES, DE, IT, SE)
iso_map = {'DEU': 'DE', 'ESP': 'ES', 'ITA': 'IT', 'SWE': 'SE'}
df['geo'] = df['geo'].replace(iso_map)

# 2. Suponemos que ya tienes df_merged con columna 'gf09_education'
#    Si no se llama exactamente así, adapta el nombre aquí:
edu_col = 'gf09_education'  # <- asegúrate de que existe

# 3. Merge
df_all = pd.merge(
    df_merged,
    df[['geo', 'time_period', 'inclusion_index']],
    on=['geo', 'time_period'],
    how='inner'
)

# 4. Calcular correlación por país
correlaciones = df_all.groupby('geo').apply(
    lambda d: d['inclusion_index'].corr(d[edu_col])
).rename("corr_inclusion_edu")

# 5. Mostrar
print("📈 Correlación entre inclusión financiera digital y % PIB en educación:")
print(correlaciones.round(3))

📈 Correlación entre inclusión financiera digital y % PIB en educación:
geo
DE    0.747
ES    0.568
IT    0.171
SE    0.409
Name: corr_inclusion_edu, dtype: float64


C:\Users\jalop\AppData\Local\Temp\ipykernel_26212\1014475658.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlaciones = df_all.groupby('geo').apply(


In [56]:
df_all

,_id,geo,time_period,gf01_general_public_services,gf07_health,gf09_education,gf1002_old_age,d51c1,d51c2,d51c3,d51d,d51e,d59,d61,d91,inclusion_index
0,685d23d139e248e523f55887,DE,2011,6.7,6.7,4.2,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.130
1,685d23d139e248e523f5588a,DE,2014,6.2,7.0,4.2,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.280
2,685d23d139e248e523f5588d,DE,2017,5.7,7.0,4.1,9.2,0.0,0.0,0.0,0.0,0.3,0.4,16.5,0.2,98.450
3,685d23d139e248e523f55891,DE,2021,6.0,8.5,4.5,9.8,0.0,0.0,0.0,0.0,0.3,0.4,17.2,0.3,99.730
4,685d23d139e248e523f558a4,ES,2011,6.2,6.5,4.4,7.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.280
5,685d23d139e248e523f558a7,ES,2014,7.1,6.1,4.1,9.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.890
6,685d23d139e248e523f558aa,ES,2017,5.5,5.9,4.0,9.1,0.0,0.0,0.0,0.0,0.0,0.4,12.2,0.5,92.125
7,685d23d139e248e523f558ae,ES,2021,5.7,7.2,4.5,10.5,0.0,0.0,0.0,0.0,0.0,0.4,13.9,0.5,97.915
8,685d23d139e248e523f558de,IT,2011,9.1,7.1,4.1,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.010
9,685d23d139e248e523f558e1,IT,2014,8.9,7.0,4.1,13.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.285


In [57]:
from bson import ObjectId

# Reconversión de tipo _id a ObjectId
df_all["_id"] = df_all["_id"].apply(lambda x: ObjectId(x) if not isinstance(x, ObjectId) else x)

# Ahora creamos el mapa correctamente
inclusion_map = {
    row["_id"]: float(row["inclusion_index"])
    for _, row in df_all.dropna(subset=["inclusion_index"]).iterrows()
}

# 2. Recorremos toda la colección y actualizamos con valor real o null
total = 0
actualizados = 0

for doc in collection.find({}, {"_id": 1}):
    doc_id = doc["_id"]
    total += 1

    inclusion_value = inclusion_map.get(doc_id, None)  # None si no está

    result = collection.update_one(
        {"_id": doc_id},
        {"$set": {"inclusion_index": inclusion_value}}
    )

    if result.modified_count:
        actualizados += 1

print(f"📦 Documentos totales recorridos: {total}")
print(f"✅ inclusion_index actualizado en: {actualizados}")

📦 Documentos totales recorridos: 146
✅ inclusion_index actualizado en: 16


In [58]:
df_all = pd.merge(df_all, df_urb, on=["geo", "time_period"], how="left")

In [60]:
df_urb

,geo,time_period,urban_rate
0,DE,2024,NaN
1,DE,2023,77.765
2,DE,2022,77.648
3,DE,2021,77.544
4,DE,2020,77.453
...,...,...,...
95,SE,2004,84.258
96,SE,2003,84.196
97,SE,2002,84.133
98,SE,2001,84.071


In [61]:
correlaciones_urban = df_all.groupby("geo").apply(
    lambda d: d["urban_rate"].corr(d["gf09_education"])
).rename("corr_urban_edu")

print(correlaciones_urban.round(3))

geo
DE    0.871
ES    0.192
IT    0.395
SE    0.723
Name: corr_urban_edu, dtype: float64


C:\Users\jalop\AppData\Local\Temp\ipykernel_26212\966206440.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  correlaciones_urban = df_all.groupby("geo").apply(


In [62]:
df_all

,_id,geo,time_period,gf01_general_public_services,gf07_health,gf09_education,gf1002_old_age,d51c1,d51c2,d51c3,d51d,d51e,d59,d61,d91,inclusion_index,urban_rate
0,685d23d139e248e523f55887,DE,2011,6.7,6.7,4.2,9.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.130,77.160
1,685d23d139e248e523f5588a,DE,2014,6.2,7.0,4.2,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.280,77.190
2,685d23d139e248e523f5588d,DE,2017,5.7,7.0,4.1,9.2,0.0,0.0,0.0,0.0,0.3,0.4,16.5,0.2,98.450,77.261
3,685d23d139e248e523f55891,DE,2021,6.0,8.5,4.5,9.8,0.0,0.0,0.0,0.0,0.3,0.4,17.2,0.3,99.730,77.544
4,685d23d139e248e523f558a4,ES,2011,6.2,6.5,4.4,7.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.280,78.673
5,685d23d139e248e523f558a7,ES,2014,7.1,6.1,4.1,9.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.890,79.366
6,685d23d139e248e523f558aa,ES,2017,5.5,5.9,4.0,9.1,0.0,0.0,0.0,0.0,0.0,0.4,12.2,0.5,92.125,80.080
7,685d23d139e248e523f558ae,ES,2021,5.7,7.2,4.5,10.5,0.0,0.0,0.0,0.0,0.0,0.4,13.9,0.5,97.915,81.056
8,685d23d139e248e523f558de,IT,2011,9.1,7.1,4.1,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.010,68.444
9,685d23d139e248e523f558e1,IT,2014,8.9,7.0,4.1,13.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.285,69.272


In [63]:
# Ahora creamos el mapa correctamente
urban_map = {
    row["_id"]: float(row["urban_rate"])
    for _, row in df_all.dropna(subset=["urban_rate"]).iterrows()
}

# 2. Recorremos toda la colección y actualizamos con valor real o null
total = 0
actualizados = 0

for doc in collection.find({}, {"_id": 1}):
    doc_id = doc["_id"]
    total += 1

    urban_value = urban_map.get(doc_id, None)  # None si no está

    result = collection.update_one(
        {"_id": doc_id},
        {"$set": {"urban_rate": urban_value}}
    )

    if result.modified_count:
        actualizados += 1

print(f"📦 Documentos totales recorridos: {total}")
print(f"✅ inclusion_index actualizado en: {actualizados}")

📦 Documentos totales recorridos: 146
✅ inclusion_index actualizado en: 146
